# EDA 2 - Time of Read Data

In [1]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

import os
import time
import shutil
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tqdm import tqdm
from PIL import Image

import sys
sys.path.append(os.path.abspath('..'))

from tools.data_reader import DataReader
from tools.decode_raw import decode_img_seg

In [2]:
dr = DataReader()

read_train_params = {
    'path': '../data/train.csv',
    'train_path': '../data/train_images',
    'height': 256,
    'width': 1600, 
    'col': False,
    'sep':'[_,]'
}

gen = dr.read_train(**read_train_params)
count = dr.count('../data/train_images')

In [ ]:
for i in tqdm(gen):
    pass

In [3]:
feature_dict = {'img': 'bytes', 'label': 'bytes'}

write_tfr_params = {
    'data_generator': gen,
    'count': count,
    'tfrpath': 'train',
    'feature_dict': feature_dict,
    'shards': 10,
    'compression': 'GZIP', 'c_level': 1}

dr.write_tfr(**write_tfr_params)

100%|█████████▉| 1255/1257 [02:12<00:00,  9.21it/s]

1667.0370497703552


In [ ]:
files = tf.train.match_filenames_once(os.path.join('..', 'tmp', 'TFRecords', 'test', '*.tfrecord'))

with tf.Session() as sess:
    # tf.local_variables_initializer().run()
    sess.run(tf.local_variables_initializer())
    print(sess.run(files))

In [ ]:
read_params = {
    'feature_dict': {'img': 'bytes', 'label': 'bytes'},
    'tfr_path': os.path.join('..', 'tmp', 'TFRecords', 'train_1', '*'),
    'decode_raw': decode_img_seg,
    'shuffle_buffer': 1000,
    'num_valid': 2000,
    'compression': 'GZIP'}

dr = DataReader()
dr.readtfrecorde(**read_params)

In [ ]:
# test tfrecords
dir_path = os.path.join('..', 'tmp', 'TFRecords', f'test')
if os.path.exists(dir_path):
    shutil.rmtree(dir_path)
os.makedirs(dir_path)

with tf.io.TFRecordWriter('../tmp/TFRecords/test/1.tfrecord') as writer:
    for i in range(100):
        img = np.zeros((4, 4)).astype('uint8').tostring()
        feature = {'img': tf.train.Feature(bytes_list=tf.train.BytesList(value=[img]))}
        example = tf.train.Example(features=tf.train.Features(feature=feature))
        writer.write(example.SerializeToString())

In [5]:
# test datasets
def parse(raw):
    features = {'img': tf.io.FixedLenFeature([], tf.string)}
    raw = tf.parse_single_example(raw, features=features)
    img = tf.decode_raw(raw['img'], tf.uint8)
    return img 

tfr_path = os.path.join('..', 'tmp', 'TFRecords', 'train', '*.tfrecord')
files = tf.io.match_filenames_once(tfr_path)
dataset = tf.data.TFRecordDataset(files, compression_type='GZIP')
dataset = dataset.map(parse)